# NER in RAG

In [1]:
%pip install pandas --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd

In [6]:
from os import environ
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

confluence_host = environ.get('CONFLUENCE_HOST')
confluence_token = environ.get('CONFLUENCE_TOKEN')
hf_token = environ.get('HF_TOKEN')
hf_write_token = environ.get('HF_WRITE_TOKEN')
gigachat_token = environ.get('GIGACHAT_TOKEN')
gigachat_pro_token = environ.get('GIGACHAT_PRO_TOKEN')
yandex_token = environ.get('YC_API_KEY')
openchat_host = environ.get('OPENCHAT_HOST')

## Датасет

### Загрузка датасета с Hugging Face

In [3]:
%pip install datasets --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from datasets import load_dataset, DatasetDict

dataset_qda = load_dataset("nizamovtimur/wikiutmn-study-gigachat")
if isinstance(dataset_qda, DatasetDict):
    train_dataset_qda = dataset_qda['train'].to_pandas()
    test_dataset_qda = dataset_qda['test'].to_pandas()
else:
    train_dataset_qda = pd.DataFrame()
    test_dataset_qda = pd.DataFrame()
len(train_dataset_qda), len(test_dataset_qda)

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(355, 67)

In [3]:
train_dataset_qda

,question,document,human_answer
0,Где продлять студак?,Для восстановления студенческого билета Вам не...,
1,Когда можно получить справку о стипендии?,"По вопросу получения справки о доходах, размер...",
2,Как перевестись на другое направление на заочке?,Заявления о переводе принимаются два раза в го...,
3,Как перевестись на другое направление?,Заявления о переводе принимаются два раза в го...,
4,Могу ли я в последний момент отказаться от сме...,"Прежде, чем выбрать элективы, рекомендуем почи...",
...,...,...,...
350,Как оформляются результаты промежуточной аттес...,"проведения промежуточной аттестации, за исключ...",
351,Кто несет ответственность за правильность офор...,"проведения промежуточной аттестации, за исключ...",
352,Что происходит с оценками после прохождения пр...,". 6. 10. Из ведомости, за исключением электрон...",
353,Кто несет ответственность за правильное внесен...,". 6. 10. Из ведомости, за исключением электрон...",


In [4]:
test_dataset_qda

,question,document,human_answer
0,Я потерял магнитную карту. К кому обратиться?,Для восстановления студенческого билета Вам не...,"Если Вы потеряли магнитную карту ( пропуск, пр..."
1,Даты подачи заявления для восстановления на очку,Заявления на восстановление в Университет по о...,Заявления на восстановление в Университет по о...
2,Что делать при потере проходки?,Для восстановления студенческого билета Вам не...,"Если Вы потеряли магнитную карту ( пропуск, пр..."
3,Я хожу в фитнес-клуб. Как заменить физкультуру?,Выбор спортивных секций по Физической культуре...,Для оформления посещения спортивного зала вмес...
4,Когда мне выдадут студенческий билет после пер...,"1. Подать заявление о переводе можно лично, об...",В течение пяти рабочих дней после поступления ...
...,...,...,...
62,как закрыть физкультуру?,Выбор спортивных секций по Физической культуре...,1 ) Посещать академические занятия ( 3 балла з...
63,когда можно поменять элективы?,"Прежде, чем выбрать элективы, рекомендуем почи...","Информация, о том, как поменять электив, отпра..."
64,как получить справку о месте учёбы?,Студенты очной формы обучения оформляют справк...,Студенты очной формы обучения оформляют справк...
65,Как можно получать баллы за физру?,Выбор спортивных секций по Физической культуре...,1 ) Посещать академические занятия ( 3 балла з...


### Генерация датасета с помощью Saiga по новым документам из боевой БД

In [17]:
# from typing import Optional
# from pgvector.sqlalchemy import Vector
# from sqlalchemy import Text, create_engine, select
# from sqlalchemy.orm import Mapped, Session, mapped_column

# Base = declarative_base()


# class Chunk(Base):
#     __tablename__ = "chunk"

#     id: Mapped[int] = mapped_column(primary_key=True)
#     confluence_url: Mapped[str] = mapped_column(Text(), index=True)
#     text: Mapped[str] = mapped_column(Text())
#     embedding: Mapped[Vector] = mapped_column(Vector(312))

#     time_created = Column(DateTime(timezone=True), server_default=func.now())
#     time_updated = Column(DateTime(timezone=True), onupdate=func.now())

# engine = create_engine(f"postgresql://{environ.get('POSTGRES_USER')}:{environ.get('POSTGRES_PASSWORD')}@{environ.get('POSTGRES_HOST')}/{environ.get('POSTGRES_DB')}", echo=False)

# with Session(engine) as session:
#     documents_from_db = [doc.text for doc in session.scalars(select(Chunk)).all()]
# len(documents_from_db)


70

In [26]:
from langchain.prompts import PromptTemplate

prompt_template = """
Действуй как студент. Составь 3 разнообразных вопроса к документу в тройных кавычках. Используй разговорный стиль речи и студенческую лексику, добавляй в вопрос название университета ТюмГУ, различные имена, названия подразделений, должности, контакты из документа. Отвечать на вопросы не нужно, напиши каждый вопрос с новой строки.

\"\"\"
{content}
\"\"\"

Вопросы:
"""

prompt = PromptTemplate.from_template(prompt_template)

In [ ]:
%pip install langchain_openai --quiet

In [27]:
from langchain_openai import ChatOpenAI

openchat = ChatOpenAI(
    # model="openchat_3.5",
    model="saiga_llama3_8b",
    openai_api_key="EMPTY",
    openai_api_base=openchat_host,
    temperature=0.6,
)

openchat_chain = prompt | openchat
openchat_chain.invoke({"content": documents_from_db[0]}).content

'1. ТюмГУ, когда я смогу оформить справку вызова на портале "Вместе", если я не сдал предыдущую сессию?\n2. Как мне узнать, что я не смогу получить справку вызова, если у меня есть академическая задолженность за предыдущую сессию?\n3. Как я смогу получить скан-копию справки вызова, если я оформил её самостоятельно на портале "Вместе"?'

In [28]:
openchat_docs = []
for doc in documents_from_db:
    query = {"content": doc}
    chat_questions = openchat_chain.invoke(query).content.strip().split("\n")
    print(chat_questions)
    for q in chat_questions:
        q = q.strip()
        if len(q) < 5:
            continue
        openchat_docs.append({
            "question": q,
            "document": doc
        })
openchat_docs = pd.DataFrame(openchat_docs)
openchat_docs

['1. Как я могу самостоятельно сформировать справку вызов на портале "Вместе"?', '2. В каком разделе и до какого срока я должен предоставить скан-копию подписанной справки вызов?', '3. При каких условиях студентам заочной формы обучения не предоставляется справка вызов на текущую сессию?']
['1. Что мне сделать, чтобы получить справку для военкомата в ТюмГУ?', '2. Где и кто мне должен помочь с получением справки для военкомата, если я студент третьего курса факультета математики?', '3. Какой контактный номер телефона и адрес местного подразделения Отдела мобилизационной подготовки, куда мне нужно обратиться для получения справки?']
['1. Какие терминалы доступны для оформления справки о подтверждении обучения для студентов очной формы обучения в ТюмГУ, и где они расположены?', '2. Что делать, если студент очной формы обучения не имеет магнитной карты, и какую информацию необходимо при себе иметь при обращении в Единой деканат?', '3. Какие адреса электронной почты и контактные данные можн

,question,document
0,1. Как я могу самостоятельно сформировать спра...,Сформировать справку - вызов Вы можете самосто...
1,2. В каком разделе и до какого срока я должен ...,Сформировать справку - вызов Вы можете самосто...
2,3. При каких условиях студентам заочной формы ...,Сформировать справку - вызов Вы можете самосто...
3,"1. Что мне сделать, чтобы получить справку для...",По вопросу получения справки для военкомата Ва...
4,2. Где и кто мне должен помочь с получением сп...,По вопросу получения справки для военкомата Ва...
...,...,...
205,2. Какие документы необходимо представить обуч...,"ФИО обучающегося, р езолюцией ответственного з..."
206,3. В документе о приоритете перехода обучающих...,"ФИО обучающегося, р езолюцией ответственного з..."
207,1. Какие условия необходимо удовлетворять обуч...,"необходимости, но не реже одного раза в семест..."
208,2. Какие приоритеты учитываются при распределе...,"необходимости, но не реже одного раза в семест..."


In [29]:
openchat_docs.to_csv("test_openchat_NE_docs.csv")

### Загрузка сгенерированного датасета

In [30]:
test_openchat_NE_docs = pd.read_csv("test_openchat_NE_docs.csv", index_col=0)
test_openchat_NE_docs

,question,document
0,1. Как я могу самостоятельно сформировать спра...,Сформировать справку - вызов Вы можете самосто...
1,2. В каком разделе и до какого срока я должен ...,Сформировать справку - вызов Вы можете самосто...
2,3. При каких условиях студентам заочной формы ...,Сформировать справку - вызов Вы можете самосто...
3,"1. Что мне сделать, чтобы получить справку для...",По вопросу получения справки для военкомата Ва...
4,2. Где и кто мне должен помочь с получением сп...,По вопросу получения справки для военкомата Ва...
...,...,...
205,2. Какие документы необходимо представить обуч...,"ФИО обучающегося, р езолюцией ответственного з..."
206,3. В документе о приоритете перехода обучающих...,"ФИО обучающегося, р езолюцией ответственного з..."
207,1. Какие условия необходимо удовлетворять обуч...,"необходимости, но не реже одного раза в семест..."
208,2. Какие приоритеты учитываются при распределе...,"необходимости, но не реже одного раза в семест..."


## Выделение именованных сущностей

импорты библиотек и функция `get_NE_library(doc: str) -> str`

### SpaCy

### Natasha

### DeepPavlov

## Векторный индекс документов

### Структура БД

In [ ]:
!docker build ../db -t db

In [3]:
!docker run --name db --rm -p 5432:5432 --mount source=virtassist_db-data,target=/var/lib/postgresql/data --env-file ../.env -d db

87b907e57fd7f79b6c28f7822f3cbce48c19f51aa1ba15124975786c0df53906


In [5]:
from typing import Optional
from pgvector.sqlalchemy import Vector
from sqlalchemy import Text, create_engine, select, text
from sqlalchemy.orm import DeclarativeBase, Mapped, Session, mapped_column

engine = create_engine(f"postgresql://{environ.get('POSTGRES_USER')}:{environ.get('POSTGRES_PASSWORD')}@{environ.get('POSTGRES_HOST')}/{environ.get('POSTGRES_DB')}", echo=False)

In [6]:
class Base(DeclarativeBase):
    pass

class Chunk(Base):
    __tablename__ = "chunk"
    id: Mapped[int] = mapped_column(primary_key=True)
    text: Mapped[str] = mapped_column(Text())
    embedding: Mapped[Optional[Vector]] = mapped_column(Vector(312))
    named_entities_lib1: Mapped[Optional[str]] = mapped_column(Text())
    named_entities_lib1: Mapped[Optional[str]] = mapped_column(Text())
    named_entities_lib1: Mapped[Optional[str]] = mapped_column(Text())

In [ ]:
with Session(engine) as session:
    session.execute(text('CREATE EXTENSION IF NOT EXISTS vector'))
    session.execute(text('DROP TABLE chunk'))
    session.commit()
Base.metadata.create_all(engine)

### Заполнение документами

In [31]:
# documents = pd.concat([train_dataset_qda["document"], test_dataset_qda["document"]]).drop_duplicates().reset_index(drop=True)
documents = test_openchat_NE_docs["document"].drop_duplicates().reset_index(drop=True)
len(documents)

70

In [ ]:
from sentence_transformers import SentenceTransformer

model_checkpoint = "nizamovtimur/rubert-tiny2-wikiutmn"
sbert_model = SentenceTransformer(model_checkpoint)
sbert_model

In [ ]:
import spacy
nlp = spacy.load('ru_core_news_lg')

with Session(engine) as session:
    for document in documents:
        doc = Chunk(
            text=document,
            embedding=sbert_model.encode(document),
            named_entities="\t".join(list(set([ent.lemma_ for ent in nlp(document).ents])))
            named_entities="\t".join(list(set([ent.lemma_ for ent in nlp(document).ents])))
            named_entities="\t".join(list(set([ent.lemma_ for ent in nlp(document).ents])))
        )
        session.add(doc)
    session.commit()

### Инициализация поиска

In [ ]:
def answer_sbert(question):
    with Session(engine) as session:
        return session.scalars(select(Chunk)
                        .order_by(Chunk.embedding.cosine_distance(
                            sbert_model.encode(question)
                            )).limit(1)).first().text


In [ ]:
# НА УРОВНЕ SQL через ORM: Именованные сущности -> эмбеддинги
# 3 отдельных метода в каждой отдельной ячейке

def answer_ner_lib(question):
    with Session(engine) as session:
        query = session.query(Chunk)
        # ЗДЕСЬ ДОЛЖНО БЫТЬ ИЛИ, А НЕ И
        for i in list(set([ent.lemma_ for ent in nlp(question).ents])):
            query = query.filter(Chunk.named_entities.contains(i))
        if len(query.all()):
            return query.order_by(Chunk.embedding.cosine_distance(
                            model.encode(question)
                            )).limit(1).first().text
        else:
            return session.scalars(select(Chunk)
                        .order_by(Chunk.embedding.cosine_distance(
                            model.encode(question)
                            )).limit(1)).first().text

answer_ner("Как перевестись из одной группы в другую в ТюмГУ")

### Выбор лучшего алгоритма

In [ ]:
test_openchat_NE_docs["sbert"] = test_openchat_NE_docs["question"].apply(answer_sbert)
test_openchat_NE_docs["lib1"] = test_openchat_NE_docs["question"].apply(answer_ner_lib)
test_openchat_NE_docs["lib2"] = test_openchat_NE_docs["question"].apply(answer_ner_lib)
test_openchat_NE_docs["lib3"] = test_openchat_NE_docs["question"].apply(answer_ner_lib)
test_openchat_NE_docs.head(5)

#### Accuracy

In [17]:
for column in test_dataset_qda.columns[3:]:
    print(column, sum(test_openchat_NE_docs[column].apply(lambda x: "" if x is None else x) == test_openchat_NE_docs.document) / len(test_openchat_NE_docs.document))

sbert_1024 0.7164179104477612
sbert_ner 0.6865671641791045


#### ROUGE-L

In [57]:
%pip install rouge --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
from rouge import Rouge
rouge = Rouge()

for column in test_openchat_NE_docs.columns[3:]:
    print(column, rouge.get_scores(test_openchat_NE_docs[column].apply(lambda x: "-" if x is None else x), test_openchat_NE_docs["document"], avg=True)['rouge-l'])

sbert_1024 {'r': 0.7860354693644203, 'p': 0.7710458235173536, 'f': 0.76751259029231}
sbert_ner {'r': 0.7751072338297648, 'p': 0.7498739222087594, 'f': 0.7497902599229568}
